# Tale Tree

In [1]:
from enum import Enum
import numpy as np

In [99]:
class Event_Type(Enum):
    QUERY = 0
    KEY = 1

class Tension_Level(Enum):
    LOW = 0
    MID = 1
    HIGH = 2

class Event():
    """ Basic building block of a narrative. """
    def __init__(self,name:str,tension_level:Tension_Level,type:Event_Type):
        self.name = name
        self.tension_level = tension_level
        self.type = type

    def __str__(self):
        return f"{self.type} | {self.tension_level} | Name: {self.name}. "
    
    def __repr__(self):
        return self.__str__()

class Timeline():
    """ Ordered sequence of Events alternating (key,query) pairs. """
    def __init__(self,timeline=None):
        if timeline:
            self.content = timeline.content.copy()
        else:
            self.content = list()

    def add_event(self,event:Event):
        """ Add an event to the existing Timeline, should be different from the previous one"""
        if not self.is_empty() and event.type == self.content[-1].type:
            raise ValueError('Trying to insert an event of same type than the previous one.')
        elif self.is_empty() and event.type != Event_Type.KEY:
            raise ValueError('The first element of a Timeline should be a KEY.')
        else:
            self.content.append(event)

    def is_empty(self) -> bool:
        """ Check if the Timeline is empty. """
        return len(self.content) == 0

    def pop_last_event(self) -> Event:
        """ Return and remove the last inserted event in the Timeline. """
        if self.is_empty():
            raise Exception('Timeline is empty')
        last_event = self.content[-1]
        self.content = self.content[:-1]
        return last_event

    def pop_first_event(self) -> Event:
        """ Remove the first inserted event in the Timeline. """
        if self.is_empty():
            raise Exception('Timeline is empty')
        first_event = self.content[0]
        self.content = self.content[1:]
        return first_event
    
    @classmethod
    def default(cls,prefix:str='a',size:int=5):
        """ Create a basic timeline with the given set of parameters. """
        timeline = cls()
        for i in range(size):
            timeline.add_event(Event(prefix+str(i),Tension_Level.MID,Event_Type.KEY))
            timeline.add_event(Event(prefix+str(i),Tension_Level.MID,Event_Type.QUERY))
        return timeline 

    def __str__(self):
        if self.is_empty():
            return 'Empty Timeline'
        return '### \t\t TIMELINE \t\t ### \n\n' + "\n".join([e.__str__() for e in self.content]) 


class Algorithm():
    """ Base class for event selection and Timelince creation. """
    def __init__(self,timelines:list):
        self.timelines = timelines
    
    def get_next_pair(self) -> tuple:
        """ Get the next pair of (key,query) selected by the algorithm. """
        return
    
    def get_sequence(self) -> Timeline:
        """ Get the full sequence derived by the algorithm. """
        return
    
class Random_Algorithm(Algorithm):
    """ Algorithm selecting randomly events for Timeline creation. """
    def __init__(self, seed, timelines):
        super().__init__(timelines)
        self.rng = np.random.default_rng(seed)

    def get_next_pair(self) -> tuple:
        candidate_timelines = [t for t in self.timelines if not t.is_empty()]
        selected_timeline = self.rng.choice(candidate_timelines)
        key = selected_timeline.pop_first_event()
        query = selected_timeline.pop_first_event()
        return key, query
    
    def get_sequence(self) -> Timeline:
        main_sequence = Timeline()
        while not all([t.is_empty() for t in self.timelines]):
            key, query = self.get_next_pair()
            main_sequence.add_event(key)
            main_sequence.add_event(query)
        return main_sequence
    
class Algorithm_Type(Enum):
    RANDOM = Random_Algorithm
    BALANCED = None # Balance to avoid sampling too much of the same tree and to keep heterogeneity across lengths of timelines
    TENSION = None


In [100]:
e = Event('test',Tension_Level.HIGH,Event_Type.QUERY)
t = Timeline.default()
rng = np.random.default_rng(0)

In [101]:
print(t)

### 		 TIMELINE 		 ### 

Event_Type.KEY | Tension_Level.MID | Name: a0. 
Event_Type.QUERY | Tension_Level.MID | Name: a0. 
Event_Type.KEY | Tension_Level.MID | Name: a1. 
Event_Type.QUERY | Tension_Level.MID | Name: a1. 
Event_Type.KEY | Tension_Level.MID | Name: a2. 
Event_Type.QUERY | Tension_Level.MID | Name: a2. 
Event_Type.KEY | Tension_Level.MID | Name: a3. 
Event_Type.QUERY | Tension_Level.MID | Name: a3. 
Event_Type.KEY | Tension_Level.MID | Name: a4. 
Event_Type.QUERY | Tension_Level.MID | Name: a4. 
